## Problem 1

In [1]:
import numpy as np
from scipy import linalg as la

def is_drazin(A, D, k):
    product1 = np.dot(A,D)
    product2 = np.dot(D,A)
    exp1 = np.linalg.matrix_power(A, k+1)
    exp0 = np.linalg.matrix_power(A, k)
    product3 = np.dot(exp1, D)
    product4 = np.dot(D,product1)
    
    if np.allclose(product1, product2) is False:
        return False
    elif np.allclose(product3, exp0) is False:
        return False
    elif np.allclose(product4, D) is False:
        return False
    else:
        return True
    

In [3]:
#Test 1

A = np.array([[1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
D = np.array([[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]])

is_drazin(A,D,1)

True

In [5]:
#Test 2

B = np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
D = np.zeros((3,3))

is_drazin(B,D,3)

True

In [7]:
#Test 3

B = np.array([[3,1,4],[5,2,6],[-2,-1,-3]])
D = np.zeros((3,3))

is_drazin(B,D,3)

False

## Problem 2

In [15]:
def drazin(A, tol):
    """Computes the Drazin inverse of an nxn matrix A"""
    n = np.shape(A)[0]
    f = lambda x: abs(x) > tol
    Q1,S,k1 = la.schur(A, sort=f)
    f = lambda x: abs(x) <= tol
    Q2,T,k2 = la.schur(A, sort=f)
    
    U = np.concatenate((S[:,:k1],T[:,:n-k1]),axis=1)
    Uinv = np.linalg.inv(U)
    V = np.dot(np.dot(Uinv, A), U)
    Z = np.zeros((n,n))
    
    if k1 != 0:
        Minv = np.linalg.inv(V[:k1,:k1])
        Z[:k1,:k1] = Minv
        
    return np.dot(np.dot(U,Z),Uinv)
    

In [20]:
#Testing Function

A = np.random.random((4,4))
D = drazin(A, 0.01)

is_drazin(A,D,2)

True

## Problem 3

In [102]:
from scipy.sparse import csgraph

def resistance(A):
    """Accepts an nxn matrix of an undirected graph
    Returns the effective resistance
    Requires the csgraph package from scipy.spare
    """
    L = csgraph.laplacian(A)
    n = np.shape(A)[0]
    R = np.zeros((n,n))
    i = 0
    while i < n:
        L_star = np.copy(L)
        identity = np.zeros(n)
        identity[i] = 1
        L_star[i] = identity
        L_drazin = drazin(L_star, 0.001)
        R[:,i] = np.diag(L_drazin)
        R[i:i] = 0
        i +=1
    return R

In [103]:
#Test Case 1: Upper Left Example
A = np.array([[1,1,0,0],[1,1,1,0],[0,1,1,1],[0,0,1,1]])
print(resistance(A))

[[ 1.  1.  2.  3.]
 [ 1.  1.  1.  2.]
 [ 2.  1.  1.  1.]
 [ 3.  2.  1.  1.]]


In [104]:
#Test Case 2: Triangle
A = np.array([[1,1,1],[1,1,1],[1,1,1]])
print(resistance(A))

[[ 1.          0.66666667  0.66666667]
 [ 0.66666667  1.          0.66666667]
 [ 0.66666667  0.66666667  1.        ]]


The output matches the test cases chosen so it appears that our function works!

## Problem 4

In [122]:
class LinkPredictor:
    def __init__(self, filename):
        
        #Initialize the list of to and from connections
        self.a = []
        self.b = []
        with open(filename, 'r') as file:
            lines = file.readlines()
        for entry in lines:
            connection = entry.split(',')
            self.a.append(connection[0])
            self.b.append(connection[1].strip())
            
        #Get the list of names
        self.names = []
        for name in self.a:
            if name not in self.names:
                self.names.append(name)
        for name in self.b:
            if name not in self.names:
                self.names.append(name)
        self.length = len(self.names)
        
        #Set up the network 
        network = []
        i = 0
        while i < self.length:
            connections = np.zeros(self.length)
            connections[i] = 1
            name = self.names[i]
            length2 = len(self.a)
            j = 0
            while j < length2:
                if self.a[j] == name:
                    partner = self.b[j]
                    partner_pos = self.names.index(partner)
                    connections[partner_pos] = 1
                j += 1
            
            length3 = len(self.b)
            k = 0
            while k < length3:
                if self.b[k] == name:
                    partner = self.b[k]
                    partner_pos = self.names.index(partner)
                    connections[partner_pos] = 1
                k += 1           
            network.append(connections)
            i += 1
            
        #Calculate connections and effective resistance
        self.A = np.array(network)
        self.A = self.A.astype(float)
        self.R = resistance(self.A)
        self.names = self.names
        